In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas numpy scikit-learn matplotlib seaborn plotly


In [3]:
import pandas as pd
import numpy as np
from google.colab import drive


drive.mount('/content/drive')


np.random.seed(42)
n = 500  


data = {
    "student_id": range(1, n+1),
    "name": [f"Student_{i}" for i in range(1, n+1)],
    "class": np.random.choice(["6A","6B","7A","7B"], n),
    "comprehension": np.clip(np.random.normal(75, 10, n), 50, 100).astype(int),
    "attention": np.clip(np.random.normal(65, 15, n), 30, 100).astype(int),
    "focus": np.clip(np.random.normal(60, 12, n), 25, 100).astype(int),
    "retention": np.clip(np.random.normal(0.6 * 75, 10, n), 20, 100).astype(int),  # correlated with comprehension
    "engagement_time": np.clip(np.random.exponential(30, n) + 10, 10, 300).astype(int)
}

df = pd.DataFrame(data)


df["assessment_score"] = (
    0.25*df["comprehension"] +
    0.25*df["attention"] +
    0.2*df["focus"] +
    0.2*df["retention"] +
    0.1*(df["engagement_time"]/3) +
    np.random.normal(0, 5, n)
).round().astype(int)


df["assessment_score"] = df["assessment_score"].clip(0, 100)


output_path = "/content/drive/MyDrive/student_learning_data"
import os
os.makedirs(output_path, exist_ok=True)

df.to_csv(f"{output_path}/students_data.csv", index=False)
df.to_json(f"{output_path}/students_data.json", orient="records")

print(f"Dataset saved to {output_path}/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset saved to /content/drive/MyDrive/student_learning_data/


In [8]:
import os
from google.colab import drive


drive.mount('/content/drive')


output_path = "/content/drive/MyDrive/student_learning_data"
os.makedirs(output_path, exist_ok=True)

df.to_csv(f"{output_path}/students_clean.csv", index=False)
df.to_json(f"{output_path}/students_clean.json", orient="records")

print(f"Saved files in {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved files in /content/drive/MyDrive/student_learning_data


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib


X = df[["comprehension","attention","focus","retention","engagement_time"]]
y = df["assessment_score"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))


model_path = f"{output_path}/student_model.pkl"
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")


Mean Squared Error: 27.958569
Model saved to /content/drive/MyDrive/student_learning_data/student_model.pkl


MSE: 27.958569


In [10]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=3, random_state=42)
df["persona"] = kmeans.fit_predict(X)


df.to_csv(f"{output_path}/students_with_persona.csv", index=False)
df.to_json(f"{output_path}/students_with_persona.json", orient="records")


print(df[["student_id","name","persona"]].head())
print(f"Updated dataset with personas saved in {output_path}")


   student_id       name  persona
0           1  Student_1        0
1           2  Student_2        2
2           3  Student_3        1
3           4  Student_4        2
4           5  Student_5        2
Updated dataset with personas saved in /content/drive/MyDrive/student_learning_data


In [11]:
print("Number of students in the dataset:", len(df))


Number of students in the dataset: 500


In [12]:
import os
from google.colab import drive


drive.mount('/content/drive')


save_path = "/content/drive/MyDrive/student_learning_data/"
os.makedirs(save_path, exist_ok=True)  


df.to_json(save_path + "student_data.json", orient="records")


persona_data = df.groupby("persona")[["comprehension", "attention", "focus", "retention", "engagement_time", "assessment_score"]].mean().reset_index()
persona_data.to_json(save_path + "persona_profiles.json", orient="records")


if "persona" in df.columns:
    df[["student_id", "name", "persona"]].to_json(save_path + "student_personas.json", orient="records")

print("All files saved in:", save_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All files saved in: /content/drive/MyDrive/student_learning_data/
